<center>
    
    Euclidean Algorithm and Auto-differentiation
    
    Author: Daniel Coble
    
    Status: Finished
</center>

This notebook accompanies a [write-up]( I made about the connection between the Euclidean Algorithm and Bézout's identity to backpropagation. I show that the calculation of solutions to Bézout's identity is no different than performing backpropagation on the graph generated by the Euclidean algorithm. Here I produce a standard implementation of the Euclidean Algorithm/Bézout's identity, one using TensorFlow, and one using forward-mode differentiation.

In [37]:
'''
A standard implementation of the Euclidean/Bezout algorithm, showing forward-mode and reverse-mode steps.
Returns a tuple (d, x, y) with d = gcd(a, b), and ax + by = d.
'''
def euclid_bezout(a, b):
    # Euclidean algorithm/forward-mode step
    C = [a, b]
    Q = []
    while(C[-1] != 0):
        q = C[-2]//C[-1]
        r = C[-2] - q*C[-1]
        Q.append(q)
        C.append(r)
    d = C[-2]
    # Bezout/revese-mode step
    x = 1
    y = -Q[-2]
    for i in range(len(Q)-3, -1, -1):
        x_new = y
        y_new = x - y*Q[i]

        x = x_new
        y = y_new
    return (d, x, y)

In [69]:
import tensorflow as tf
import numpy as np

'''
A TensorFlow implementation of the Euclidean algorithm. Backpropagation produces the Bezout values x and y.
Returns a tuple (d, x, y) with d = gcd(a, b) and ax + by = d
'''
def tensorflow_bezout(a, b):
    a = tf.Variable(a, dtype=tf.float32)
    b = tf.Variable(b, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch([a, b])
        c1 = a
        c2 = b
        while(c2 > 0):
            q = c1 // c2
            c1_new = c2
            c2_new = c1 - q*c2
            
            c1 = c1_new
            c2 = c2_new
        d = c1
    [x, y] = tape.gradient(d, [a, b])
    return (int(d.numpy()), int(x.numpy()), int(y.numpy()))

In [84]:
'''
A class which represents tuple numbers (c, dc/da, dc/db)
'''
class TupleNum():
    
    def __init__(self, num):
        self.num = num
    
    def __mul__(self, other):
        c = self.num[0]*other.num[0]
        dcda = self.num[1]*other.num[0] + self.num[0]*other.num[1]
        dcdb = self.num[2]*other.num[0] + self.num[0]*other.num[2]
        return TupleNum((c, dcda, dcdb))
    
    def __add__(self, other):
        c = self.num[0] + other.num[0]
        dcda = self.num[1] + other.num[1]
        dcdb = self.num[2] + other.num[2]
        return TupleNum((c, dcda, dcdb))
    
    def __sub__(self, other):
        c = self.num[0] - other.num[0]
        dcda = self.num[1] - other.num[1]
        dcdb = self.num[2] - other.num[2]
        return TupleNum((c, dcda, dcdb))
    
    def __floordiv__(self, other):
        c = self.num[0]//other.num[0]
        return TupleNum((c, 0, 0))

'''
This forward implementation of Euclidean algorithm doesn't contain a reverse Bezout step, but produces the values via forward
mode autodifferentation.
Returns a tuple (d, x, y) with d = gcd(a, b) and ax + by = d
'''
def forward_bezout(a, b):
    c1 = TupleNum((a, 1, 0))
    c2 = TupleNum((b, 0, 1))
    while(c2.num[0] > 0):
        q = c1 // c2
        c1_new = c2
        c2_new = c1 - q*c2
        
        c1 = c1_new
        c2 = c2_new
    d = c1
    return d.num

In [36]:
# test each implementation
print(euclid_bezout(314, 159))
print(tensorflow_bezout(314, 159))
print(forward_bezout(314, 159))

(1, -40, 79)
